# Pandas Quirks

Pandas is a very powerful tool that we will use a lot in this course.
However, just like any other framework, Pandas also has its own quirks that you will have to get used to.

Once you start to understand the "Pandas' way" of doing things, things will start to feel pretty intuitive.
But getting to that point can be hard.
So in this example we will discuss some Pandas terminology, mechanisms, and quirks.
Hopefully understanding these will help you think in the "Pandas' way".

(But coming from a database perspective, Pandas is bananas. -eriq)

Let's start out with some simple data:

In [ ]:
import pandas

surf_data = {
    'Name': ['Andrew', 'Eriq', 'Reilly', 'Michael'],
    'Surfing Score': [5.3, 5.9, 5.1, 5.2],
    'Qualification': ['MS', 'PhD', 'PhD', 'PhD'],
}

surf_frame = pandas.DataFrame(surf_data)
surf_frame

## Indexes

In Pandas, every frame (and series) has a special column called the "index".
(Note that this column is not related to database indexes or the concept of "indexing" or selecting data.)
The [index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.index.html) is used to uniquely identify every row (think "primary key" in databases).
By default, the index is a special column of ints (a range of ints actually) that sits right next to the other columns, but cannot be selected the same way.
If you look at the `surf_frame` above, you can see the index column on the far left.

You can select the index column using `.index`:

In [ ]:
surf_frame.index

You can also use `.index` to assign a new index to your frame.
As long as what you assign the index has the right number of elements.
Ideally, each element should also be unique and hashable (otherwise other Pandas operations can mess up down the road).

In [ ]:
test_frame = surf_frame.copy()
test_frame.index = reversed(range(100, 104))
test_frame

In [ ]:
test_frame = surf_frame.copy()
test_frame.index = ["A", 1, ("Dog", "Hat", 1), 5.5]
test_frame

## Axes

There are many different operations you can do on a Pandas frame.
Sometimes it is obvious that the operation should be on the frame, a row, a col, or a cell.
But sometimes operations could make sense in multiple directions,
like if you wanted to get the number of unique values ([pandas.DataFrame.nunique](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html)).
You may want the number of unique values in a row or in a column.
To resolve this, Pandas uses the concept of "axes".
Where `axis = 0` normally means along rows (or along the index) and `axis = 1` normally means along columns.
`axis = 0` is usually the default.

Therefore, to find the number of unique values **going through each row**, we can use:

In [ ]:
surf_frame.nunique(axis = 0)

The above operation took each column and **went along the row axis** to compute the number of unique values.

To find the number of unique values **going through each column**, we can use:

In [ ]:
surf_frame.nunique(axis = 1)

Note that in the operation along the 0th axis (the rows), we got to see the column names.
When going through the 1st axis (the cols), we get to see the index value for each row.
As we say before we can change the index and see the result in the labels:

In [ ]:
test_frame = surf_frame.copy()
test_frame.index = ["A", "B", "C", "D"]
test_frame.nunique(axis = 1)

## Data Types

In Pandas, data is (almost) always in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)
or a [pandas.Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).
If an operation is guaranteed to only have one column, then the data will be in a series otherwise it will be in a frame.

Thankfully, many methods exist for both data types so many people don't even realise when they are using one or the other.
Not knowing the underlying data type you are using is fine for simple operations.
But if you are doing more complex things, you may need to know what data type you are working with.

For example, if we select a single column from a frame we will get a series back:

In [ ]:
value = surf_frame['Name']

print(type(value))
value

And if we select multiple columns from a frame, then we will get a frame back:

In [ ]:
value = surf_frame[['Name', 'Qualification']]

print(type(value))
value

However, be careful and always pay attention to what an operation **could** return.
When we use a list to select columns from a frame, then we **could** select multiple columns.
Therefore, this will always return a frame even through only one column is selected and the result can fit into a series.

In [ ]:
value = surf_frame[['Name']]

print(type(value))
value

The difference between these two is just a pair of brackets,
and you can see how people can easily mess this up and get in trouble.
```
surf_frame['Name']
surf_frame[['Name']]
```

## Overriding Operations

One of the ways that Pandas tries to make their framework more intuitive is to [override existing Python operators](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types).
This allows you do do things like use the `+` operator to add a constant to all rows in a column:

In [ ]:
display(surf_frame['Surfing Score'])
print('---')
display(surf_frame['Surfing Score'] + 1)

You can even use the `=` operator to assign that new score back into the frame:

In [ ]:
test_frame = surf_frame.copy()
test_frame['Surfing Score'] = test_frame['Surfing Score'] + 1
test_frame

This can help make your code more readable, especially for people not familiar with Pandas.
However, it can get tricky as you do more advanced operations.
Specifically, you will sometimes need to know if you are using a native Python operator or if you are using an overridden Pandas operator.
The following example will walk you through a specific use case and hopefully you can build off that to help you in identifying what operators you are using.

### Boolean Selection

Consider the following data and frame.

In [ ]:
bool_data = {
    'col-all-bool': [True, False, True],
    'col-missing-values': [True, False, None],
    'col-mixed-values': [True, False, 1],
}

bool_frame = pandas.DataFrame(bool_data)
bool_frame

All three columns have boolean values (`True`/`False`), but some the columns have some quirks:
 - `col-all-bool` -- All values are boolean.
 - `col-missing-values` -- Has boolean values and a `None` to represent a missing value (which is very common). 
 - `col-mixed-values` -- Has boolean values as well as a `1`, which is commonly used to represent a true value (especially in machine learning).

This frame has reasonable data that you could see a variety of settings.

If we wanted to select the true/false values from this frame, we could use native Pandas operators like:
```
# True Values
frame[frame['column-name']]

# False Values
frame[~frame['column-name']]
```

Or, we could use native Python operators like:
```
# True Values
frame[frame['column-name'] == True]

# False Values
frame[frame['column-name'] == False]
```

Note that this second set will use the Python `==` operator and Python representations for `True` and `False`.
Let's make a function to see how these different methods work on our data.

In [ ]:
def test_boolean_selection(frame, column_name):
    # Use the Pandas method.
    try:
        print("True Pandas Values for '%s': %s" % (column_name, list(frame[frame[column_name]][column_name])))
        print("False Pandas Values for '%s': %s" % (column_name, list(frame[~frame[column_name]][column_name])))
    except Exception as ex:
        print("Got an exception while trying to access the Pandas way: ", ex)

    # Use the Python method.
    try:
        print("True Python Values for '%s': %s" % (column_name, list(frame[frame[column_name] == True][column_name])))
        print("False Python Values for '%s': %s" % (column_name, list(frame[frame[column_name] == False][column_name])))
    except Exception as ex:
        print("Got an exception while trying to access the Python way: ", ex)

Now let's try this function out on our cleanest column:

In [ ]:
test_boolean_selection(bool_frame, 'col-all-bool')

When all the values are booleans, we get the same functionality no matter how we access the data.

Now what happens when we have missing data:

In [ ]:
test_boolean_selection(bool_frame, 'col-missing-values')

Now things start to get concerning when our data is not so clean (which is always the case in data science).
When there is a missing value (`None`), then the Pandas way throws an exception and the Python way just doesn't count the value as either true or false.

So, which way is correct here?

There is actually no correct answer here.
It all depends on the data and the situation.
Generally, it is usually safer to use Pandas operations when you are dealing with Pandas data, but this is just a rule-of-thumb.
You may need to guarantee you code doesn't throw an exception even if the data is wrong,
and in that case the Python way may be safer.

(Personally I generally prefer the Pandas way here,
since this could be a sign of poor data cleaning and I generally want to be alerted (via a raised exception) when my data is wrong. -eriq)

Finally, what happens when we have mixed values:

In [ ]:
test_boolean_selection(bool_frame, 'col-mixed-values')

Given Pandas' behavior in the last test, it makes sense that it also throws an exception here.

The behavior that may really catch you off guard here is the Python behavior.
The integer 1 is counted as a true value.
Instead of using hard truth values, Python uses the concept of "Truthy" or things that are like true values.
Therefore, there may be some things that you don't expect to match when using Python operators.

In [ ]:
values = [
    True, False,
    0, 1, -1,
    '', 'a',
    [], [1], ['a'], [True], [False],
]

for value in values:
    print("Value: %s,\tIs T?: %s,\tIs F?: %s" % (value, (value == True), (value == False)))

Most of these are not too surprising.
But if you want to be surprised try this test in JavaScript.